# Brain Tumor Detection
## Single Task Validation - Building multi-lable binary Classifier
Description
This dataset was originally created by Yousef Ghanem. To see the current project, which may have been updated since this version, please go here: https://universe.roboflow.com/yousef-ghanem-jzj4y/brain-tumor-detection-fpf1f.

This dataset is part of RF100, an Intel-sponsored initiative to create a new object detection benchmark for model generalizability.

Access the RF100 Github repo: https://github.com/roboflow-ai/roboflow-100-benchmark

## Imports

In [1]:
# Go to project root folder
import os
os.chdir("../")
%pwd

'/workspaces/brain-tumor-detection'

In [2]:
from tqdm.notebook import tqdm
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

import tensorflow as tf
tf.random.set_seed(42)

import matplotlib.pyplot as plt

2025-03-13 15:07:19.588439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741878439.596269  340365 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741878439.598635  340365 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1741878439.607287  340365 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1741878439.607296  340365 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1741878439.607297  340365 computation_placer.cc:177] computation placer alr

In [3]:
found_gpu = tf.config.list_physical_devices('GPU')
if not found_gpu:
    raise Exception("No GPU found")
found_gpu, tf.__version__

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.19.0')

In [4]:
from src.data_handler.data_loader import DataLoader
from src.data_handler.annotation_processor import AnnotationProcessor
from src.data_handler.preprocessor import Preprocessor

In [5]:
# auto reload dotenv 
%load_ext dotenv
%dotenv

# auto reload libs
%load_ext autoreload
%autoreload 2

## Paths Setup

In [6]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/brain-tumor-2/train/


In [7]:
cfg.DATASET_DIRS

{'TRAIN_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/train/', 'VALIDATION_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/valid', 'TEST_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/test'}

In [8]:
DATASET_DIRS = Path(cfg.DATASET.DATASET_DIR)
TRAIN_DIR = Path(cfg.DATASET_DIRS.TRAIN_DIR)
VALIDATION_DIR = Path(cfg.DATASET_DIRS.VALIDATION_DIR)
TEST_DIR = Path(cfg.DATASET_DIRS.TEST_DIR)


IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE
LOG_DIR = cfg.OUTPUTS.LOG_DIR
CHECK_POINT_DIR = Path(cfg.OUTPUTS.CHECKPOINT_PATH)
CLASS_NAME = [
    'label0',
    'label1',
    'label2'
]
class_map = {k: v for k, v in enumerate(CLASS_NAME)}

NUM_EPOCHS = cfg.TRAIN.NUM_EPOCHS
LEARNING_RATE = cfg.TRAIN.LEARNING_RATE

NUM_CLASSES = len(CLASS_NAME)


## Dataset Download from Roboflow

In [9]:
if not TRAIN_DIR.exists():
    from roboflow import Roboflow
    rf = Roboflow()
    project = rf.workspace("roboflow-100").project("brain-tumor-m2pbp")
    version = project.version(2)
    dataset = version.download("tensorflow")      

## Load images from directory

### Load Training datasets

In [10]:
prepare_train_dataset = AnnotationProcessor(annotation_file= str(TRAIN_DIR/'_annotations.csv'))
_class_map = {v: k for k, v in enumerate(CLASS_NAME)}
train_images, train_class_ids, train_bboxes  = prepare_train_dataset.process_annotations(image_dir=TRAIN_DIR, class_id_map=_class_map)

len(train_images), len(train_class_ids), len(train_bboxes)

(6930, 6930, 6930)

In [11]:
train_images[0],train_class_ids[0], train_bboxes[0]

('datasets/brain-tumor-2/train/volume_337_slice_89_jpg.rf.63cc21fc850bfb89383c90a49ece9826.jpg',
 [0, 1, 2],
 array([[0.57916667, 0.33333333, 0.75833333, 0.425     ],
        [0.5375    , 0.275     , 0.82916667, 0.5125    ],
        [0.57083333, 0.32083333, 0.76666667, 0.45416667]]))

In [12]:
train_dl = DataLoader(train_images, train_class_ids, train_bboxes)
train_ds = train_dl.load_train_dataset()
train_ds = Preprocessor(train_ds).preprocess()
train_ds = train_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

I0000 00:00:1741878453.705705  340365 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6901 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6


In [13]:
for batch in train_ds.take(1):
    image, cls = batch
    print(image.shape, cls.shape)
    print(cls[5])
    print(image[1].numpy().min(), image[1].numpy().max())
    for c in cls:
        print(c.numpy())

(32, 240, 240, 3) (32, 3)
tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)
-114.57817 150.90729
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 0.]


2025-03-13 15:07:40.274637: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Validation datasets 

In [14]:
prepare_valid_dataset = AnnotationProcessor(annotation_file= str(VALIDATION_DIR/'_annotations.csv'))

valid_image_paths, valid_class_ids, valid_bboxes  = prepare_valid_dataset.process_annotations(image_dir=VALIDATION_DIR, class_id_map=_class_map)
len(valid_image_paths), len(valid_class_ids), len(valid_bboxes)

(1980, 1980, 1980)

In [15]:
valid_dl = DataLoader(valid_image_paths, valid_class_ids, valid_bboxes).load_val_dataset()
valid_ds = Preprocessor(valid_dl).preprocess()
valid_ds = valid_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

In [16]:
for batch in valid_ds.take(1):
    image, cls, = batch
    print(cls.shape)
    print(image.shape)
    print(image[1].numpy().min(), image[1].numpy().max())

(32, 3)
(32, 240, 240, 3)
-123.68 147.061


2025-03-13 15:07:41.806101: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Training Setup

In [17]:
METRICS = [
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC', multi_label=True), 
    tf.keras.metrics.F1Score(name='f1_score',average='weighted'),
]

### Define  Callbacks

In [18]:
import os

to_monitor = 'val_loss'
mode = 'min'
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, 
                                            patience=5, 
                                            monitor=to_monitor,
                                            mode=mode,
                                            min_lr=1e-6,
                                            verbose=1),

    tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(str(CHECK_POINT_DIR), "classifier_ckpt_{epoch}.keras") ,
                                        save_weights_only=False,
                                        save_best_only=True,
                                        monitor=to_monitor,
                                        mode=mode,
                                        verbose=1),
                                        
    # tf.keras.callbacks.EarlyStopping(monitor=to_monitor, 
    #                                 patience=10,
    #                                 mode=mode, 
    #                                 restore_best_weights=True),

]

### Define Optimizer

In [21]:
optimizer=tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE, momentum=0.9)

## Define loss

In [22]:
padded_class_ids, _ = train_dl.pad_cls_id_bbx()
padded_class_ids

array([[0, 1, 2],
       [1, 0, 0],
       [0, 1, 2],
       ...,
       [0, 1, 2],
       [0, 1, 2],
       [0, 1, 2]], dtype=int32)

In [23]:
from sklearn.preprocessing import Binarizer
transformer = Binarizer().fit(padded_class_ids) 
bin_cls_ids = transformer.transform(padded_class_ids)
bin_cls_ids

array([[0, 1, 1],
       [1, 0, 0],
       [0, 1, 1],
       ...,
       [0, 1, 1],
       [0, 1, 1],
       [0, 1, 1]], dtype=int32)

In [24]:
from src.losses import binary_weighted_loss as _loss

positive_weights, negative_weights = _loss.compute_class_weights(bin_cls_ids)
positive_weights, negative_weights 

(array([0.62597403, 0.31861472, 0.47417027]),
 array([0.37402597, 0.68138528, 0.52582973]))

## Define ResNet50 Model Classifier

In [25]:
from src.models.resnet50 import resnet50_classifier
tf.keras.backend.clear_session()
model = resnet50_classifier(input_shape=(IMG_SIZE,IMG_SIZE,3), num_classes=NUM_CLASSES)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 246, 246,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 120, 120,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 120, 120,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 120, 120,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 122, 122,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 60, 60,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 60, 60,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 60, 60,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 60, 60,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 60, 60,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 60, 60,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 60, 60,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 60, 60,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 60, 60,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 60, 60,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 60, 60,    │      1,024 │ conv2_block1_3_c

 Total params: 24,902,787 (95.00 MB)

 Trainable params: 24,849,155 (94.79 MB)

 Non-trainable params: 53,632 (209.50 KB)

## Model Building and Compilation

In [26]:
model.compile(
    optimizer=optimizer,
    # loss= tf.keras.losses.BinaryCrossentropy(from_logits=False),
    loss= _loss.set_binary_crossentropy_weighted_loss(positive_weights, negative_weights),
    metrics=METRICS)  # Use IoU metric

## Train and Validate the model

In [27]:
EPOCHS = 50

In [ ]:
import mlflow

mlflow.set_experiment("/brain-tumor-resnet50_classifier")
mlflow.tensorflow.autolog(log_models=True, log_datasets=False)

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    batch_size=BATCH_SIZE,
    callbacks=[callbacks],
)

2025/03/13 15:09:53 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.18.0, but the installed version is 2.19.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.
2025/03/13 15:09:53 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'fcc33068503a41e2b097d23fbf2aeda7', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/50


I0000 00:00:1741878608.914626  340606 service.cc:152] XLA service 0x7e3f34003110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741878608.914692  340606 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2025-03-13 15:10:09.373638: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741878610.666164  340606 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1741878624.512166  340606 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - AUC: 0.3343 - f1_score: 0.4593 - loss: 11.9511 - precision: 0.8441 - recall: 0.5706
Epoch 1: val_loss improved from inf to 11.87546, saving model to output/checkpoints/classifier_ckpt_1.keras


217/217 ━━━━━━━━━━━━━━━━━━━━ 78s 216ms/step - AUC: 0.3344 - f1_score: 0.4593 - loss: 11.9510 - precision: 0.8441 - recall: 0.5706 - val_AUC: 0.3639 - val_f1_score: 0.4973 - val_loss: 11.8755 - val_precision: 0.8849 - val_recall: 0.8500 - learning_rate: 1.0000e-05
Epoch 2/50
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - AUC: 0.3680 - f1_score: 0.4817 - loss: 11.9028 - precision: 0.8619 - recall: 0.6359
Epoch 2: val_loss improved from 11.87546 to 11.80076, saving model to output/checkpoints/classifier_ckpt_2.keras


217/217 ━━━━━━━━━━━━━━━━━━━━ 33s 130ms/step - AUC: 0.3680 - f1_score: 0.4817 - loss: 11.9027 - precision: 0.8619 - recall: 0.6359 - val_AUC: 0.3171 - val_f1_score: 0.4233 - val_loss: 11.8008 - val_precision: 0.9267 - val_recall: 0.7996 - learning_rate: 1.0000e-05
Epoch 3/50
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - AUC: 0.3302 - f1_score: 0.4911 - loss: 11.8652 - precision: 0.8617 - recall: 0.6845
Epoch 3: val_loss improved from 11.80076 to 11.69253, saving model to output/checkpoints/classifier_ckpt_3.keras


217/217 ━━━━━━━━━━━━━━━━━━━━ 34s 131ms/step - AUC: 0.3301 - f1_score: 0.4912 - loss: 11.8652 - precision: 0.8617 - recall: 0.6845 - val_AUC: 0.3403 - val_f1_score: 0.5031 - val_loss: 11.6925 - val_precision: 0.9175 - val_recall: 0.8913 - learning_rate: 1.0000e-05
Epoch 4/50
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - AUC: 0.3321 - f1_score: 0.4971 - loss: 11.8284 - precision: 0.8676 - recall: 0.7375
Epoch 4: val_loss improved from 11.69253 to 11.52009, saving model to output/checkpoints/classifier_ckpt_4.keras


217/217 ━━━━━━━━━━━━━━━━━━━━ 31s 120ms/step - AUC: 0.3322 - f1_score: 0.4971 - loss: 11.8283 - precision: 0.8676 - recall: 0.7375 - val_AUC: 0.3877 - val_f1_score: 0.4987 - val_loss: 11.5201 - val_precision: 0.8949 - val_recall: 0.9732 - learning_rate: 1.0000e-05
Epoch 5/50
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - AUC: 0.3633 - f1_score: 0.5063 - loss: 11.7904 - precision: 0.8766 - recall: 0.7772
Epoch 5: val_loss improved from 11.52009 to 11.45584, saving model to output/checkpoints/classifier_ckpt_5.keras


## Training Visualization

In [ ]:
import matplotlib.pyplot as plt

def visualize_training_results(history):
    """
    Visualizes training and validation loss, and training and validation accuracy.

    Args:
        history: A dictionary or object containing training history data.
                 For example, a Keras History object or a dictionary with keys:
                 'loss', 'val_loss', 'accuracy', 'val_accuracy'.
    """

    if isinstance(history, dict):
        # Assumes history is a dictionary
        loss = history.get('loss')
        val_loss = history.get('val_loss')
        accuracy = history.get('accuracy')
        val_accuracy = history.get('val_accuracy')
    else:
        # Assumes history is a Keras History object or similar
        loss = history.history.get('loss')
        val_loss = history.history.get('val_loss')
        accuracy = history.history.get('accuracy')
        val_accuracy = history.history.get('val_accuracy')

    if loss and val_loss:
        epochs = range(1, len(loss) + 1)

        plt.figure(figsize=(12, 5))

        # Plot training & validation loss values
        plt.subplot(1, 2, 1)
        plt.plot(epochs, loss, 'r', label='Training loss')
        plt.plot(epochs, val_loss, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

    if accuracy and val_accuracy:
        if not (loss and val_loss):
          plt.figure(figsize=(12, 5))
        else:
          plt.subplot(1, 2, 2)
        # Plot training & validation accuracy values
        plt.plot(epochs, accuracy, 'r', label='Training accuracy')
        plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
        plt.title('Training and validation accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

    plt.tight_layout() #prevents overlapping titles/labels
    plt.show()


In [ ]:
visualize_training_results(history.history)

## Model Evaluation

### Testing Datasets setup

In [ ]:

prepare_test_dataset = AnnotationProcessor(annotation_file= str(TEST_DIR/'_annotations.csv'))
_class_map = {v: k for k, v in enumerate(CLASS_NAME)}
test_image_paths, test_class_ids, test_bboxes = prepare_test_dataset.process_annotations(image_dir=TEST_DIR, class_id_map=_class_map)

len(test_image_paths), len(test_class_ids), len(test_bboxes)

In [ ]:
test_dl = DataLoader(test_image_paths, test_class_ids, test_bboxes)
test_ds = test_dl.load_val_dataset()
test_padded_class_ids = test_dl.pad_cls_id_bbx()[0]
test_ds = Preprocessor(test_ds).preprocess()
test_ds = test_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

In [ ]:
results = model.evaluate(test_ds, return_dict=True, steps=1)
print("Testing accuracy: ", results)

In [ ]:
results

In [ ]:
import numpy as np
y_true = np.array([y.astype(int) for _, y in test_ds.unbatch().as_numpy_iterator()])
y_true

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_ds)

In [ ]:
y_pred = (y_pred>0.5).astype(int)
y_pred

In [ ]:
print(classification_report(y_true, y_pred, labels=[0,1,2], target_names=CLASS_NAME))

In [ ]:
from sklearn import metrics

auc_roc_values = []
fig, axs = plt.subplots(1)
for i in range(len(CLASS_NAME)):
    try:
        roc_score_per_label = metrics.roc_auc_score(y_true=y_true[:,i], y_score=y_pred[:,i])
        auc_roc_values.append(roc_score_per_label)
        fpr, tpr, _ = metrics.roc_curve(y_true=y_true[:,i],  y_score=y_pred[:,i])
        
        axs.plot([0,1], [0,1], 'k--')
        axs.plot(fpr, tpr, 
                label=f'{CLASS_NAME[i]} - AUC = {round(roc_score_per_label, 3)}')

        axs.set_xlabel('False Positive Rate')
        axs.set_ylabel('True Positive Rate')
        axs.legend(loc='lower right')
    except:
        print(
            f"Error in generating ROC curve for {CLASS_NAME[i]}. "
            f"Dataset lacks enough examples."
        )
plt.savefig(f"{cfg.OUTPUTS.OUPUT_DIR}/ROC-Curve.png")
mlflow.log_figure(fig, 'ROC-Curve.png')
results = model.evaluate(test_ds, verbose=0,return_dict=True)
mlflow.log_metrics(results)